# ライブラリのインポート

In [2]:
from qiskit import *
from qiskit.quantum_info import Operator, Statevector, DensityMatrix
import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt

%matplotlib inline

# パラメータ

In [ ]:
num = 6
t = 3
M = 100
h = 3

delta = t/M

In [ ]:
# オブザーバブルの準備
array_z = np.eye(2**num)
for i in range(1,2**num,2):
    array_z[i][i]=-1
magnetization_obs = Operator(array_z)

# //ーーーーーーーーーーーーーーーーーーーーー
# 初期状態ベクトルの準備
state = Statevector.from_label('0' * num)

# //ーーーーーーーーーーーーーーーーーーーーー
# Z_i*Z_{i+1}の行列表示
zz_matrix = np.array([[1,0,0,0],
                      [0,-1,0,0],
                      [0,0,-1,0],
                      [0,0,0,1]])

# 横磁場の行列表現
hx_matrix = h*np.array([[0,1], 
                        [1,0] ] )

# 相互作用と横磁場のハミルトニアンを作成
zz_terms = [Operator.from_label('I' * i + 'ZZ' + 'I' * (num - i - 2)) for i in range(num - 1)]
hx_terms = [Operator.from_label('I' * i + 'X' + 'I' * (num - i - 1)) for i in range(num)]

# ハミルトニアンの総和を取る
zz_hamiltonian = sum(zz_terms)
hx_hamiltonian = sum(hx_terms)
hamiltonian = zz_hamiltonian + hx_hamiltonian

# ハミルトニアンの行列表現を取得
ham_matrix = hamiltonian.to_matrix()

# ハミルトニアンの対角化
eigenvalues, eigenvectors = np.linalg.eigh(ham_matrix)

# e^{-iH*delta} の行列表現を作成
e_iHdelta = expm(-1j * hamiltonian * delta).data

# 回路に変換
qc = QuantumCircuit(num)
qc.unitary(e_iHdelta, range(num), label='e^{-iHt}')

# //ーーーーーーーーーーーーーーーーーーーーー
x = [i*delta for i in range(M+1)]
y = []

expectation_value = state.expectation_value(magnetization_obs).real
y.append(expectation_value)

for i in range(M):
    state = state.evolve(qc)
    expectation_value = state.expectation_value(magnetization_obs).real
    y.append(expectation_value)

plt.xlabel("Time (t)")
plt.ylabel("Value of magnetization")
plt.title("Dynamics of Ising model")
plt.plot(x, y, "-")
plt.show()